<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/03_xyz_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-07-21 15:35:00--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11848 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.57K  --.-KB/s    in 0s      

2022-07-21 15:35:01 (111 MB/s) - ‘utils.py’ saved [11848/11848]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import ModelCreator, get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.4 MB/s 


In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'data')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 20092 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
xyz_creator = ModelCreator(HUB_URL, 'XYZ-model')
xyz_model = xyz_creator.make_model(img_size=IMG_SIZE)

Model: "XYZ-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [10]:
callbacks = xyz_creator.get_callbacks()

In [11]:
history = xyz_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        class_weight=class_weight,
                        validation_data=val_ds)

Epoch 1/15
628/628 [==============================] - 3314s 5s/step - loss: 12.9164 - auc: 0.8271 - accuracy: 0.7393 - val_loss: 0.7705 - val_auc: 0.8378 - val_accuracy: 0.7502
Epoch 2/15
628/628 [==============================] - 308s 488ms/step - loss: 7.3325 - auc: 0.9251 - accuracy: 0.8468 - val_loss: 0.7362 - val_auc: 0.8626 - val_accuracy: 0.7871
Epoch 3/15
628/628 [==============================] - 300s 475ms/step - loss: 5.5115 - auc: 0.9505 - accuracy: 0.8825 - val_loss: 0.8049 - val_auc: 0.8497 - val_accuracy: 0.7728
Epoch 4/15
628/628 [==============================] - 300s 475ms/step - loss: 3.9943 - auc: 0.9708 - accuracy: 0.9152 - val_loss: 0.7470 - val_auc: 0.8664 - val_accuracy: 0.7969
Epoch 5/15
628/628 [==============================] - 302s 477ms/step - loss: 2.9560 - auc: 0.9813 - accuracy: 0.9355 - val_loss: 0.7381 - val_auc: 0.8810 - val_accuracy: 0.8126
Epoch 6/15
628/628 [==============================] - 301s 476ms/step - loss: 2.0470 - auc: 0.9892 - accuracy: 

In [12]:
!cp ./logs/checkpoints/XYZ-model.h5 ./drive/MyDrive/checkpoints/xyz-model-aug.h5